In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from matplotlib.ticker import PercentFormatter
import plotly.graph_objects as go
import plotly.express as px
import geopandas as gpd
import skmob
from skmob.preprocessing import detection

In [40]:
ventas_tri1_df = pd.read_csv("../../data/output/tri1.zip")
ventas_tri2_df = pd.read_csv("../../data/output/tri2.zip")
ventas_tri3_df = pd.read_csv("../../data/output/tri3.zip")
ventas_tri4_df = pd.read_csv("../../data/output/tri4.zip")
hiper_df = pd.read_csv("../../data/output/hiper_out.zip")

In [41]:
ventas_tri1_df.head()

,fecha,hora,grupo,cadena,comercio,proveedor,producto,medio,total,cliente_final,saldo_despues
0,2019-01-03,10:34:48,408,1632,58553,83,2,78,1.00,%...&*.'!.,48730.39
1,2019-01-31,14:08:53,25,205,43917,5,15,3,1.00,%...&*%%+.,658.64
2,2019-01-04,08:18:08,25,338,47039,144,2,78,52.64,).%..-,1561.34
3,2019-01-03,07:53:27,25,1034,20011,6,2,3,10.00,%.+&(&+%)*,622.69
4,2019-01-18,14:13:30,4,8,30240,5,15,78,1.00,%.+.-)!!&),168.44


In [42]:
ventas_tri1_df_test = ventas_tri1_df.loc[ventas_tri1_df['cliente_final'] == "%.-&&!'.++"]
ventas_tri1_df_test['comercio'].value_counts()

90567    32
92176     5
39969     1
46624     1
39177     1
Name: comercio, dtype: int64

In [43]:
hiper_df.head()

,fecha_ingreso,grupo,cadena,comercio,estado,pais,region,provincia,ciudad,zona,ejecutivo,latitud,longitud
0,2019-01-02,4,4,90090,1,1.0,2.0,1542.0,1564.0,7.0,326.0,-3.327939,-79.807340
1,2019-01-02,62,266,90088,BT,1.0,2.0,5.0,27.0,33.0,20.0,0.000000,0.000000
2,2019-01-02,4,4,90091,BT,1.0,2.0,4.0,45.0,26.0,367.0,-1.715146,-79.864203
3,2019-01-02,168,647,90089,0,1.0,2.0,973.0,1017.0,33.0,20.0,0.000000,0.000000
4,2019-01-03,88,381,90100,1,1.0,3.0,968.0,1409.0,13.0,349.0,0.000000,0.000000


In [44]:
test = hiper_df.loc[hiper_df['comercio'] == 90567];
test

,fecha_ingreso,grupo,cadena,comercio,estado,pais,region,provincia,ciudad,zona,ejecutivo,latitud,longitud
162,2019-01-28,25,197,90567,BT,1.0,2.0,4.0,24.0,2.0,373.0,-2.202628,-79.926652
2604,2019-01-28,25,197,90567,BT,1.0,2.0,4.0,24.0,2.0,373.0,-2.202628,-79.926652


In [45]:
hiper_df = hiper_df.drop_duplicates(keep='first')

In [46]:
test = hiper_df.loc[hiper_df['comercio'] == 90567];
test

,fecha_ingreso,grupo,cadena,comercio,estado,pais,region,provincia,ciudad,zona,ejecutivo,latitud,longitud
162,2019-01-28,25,197,90567,BT,1.0,2.0,4.0,24.0,2.0,373.0,-2.202628,-79.926652


In [47]:
hiper_df["fecha_ingreso"] = pd.to_datetime(
    hiper_df["fecha_ingreso"].astype(str), format="%Y-%m-%d")

In [48]:
gdf_hiper = gpd.GeoDataFrame(hiper_df, geometry=gpd.points_from_xy(
    hiper_df["longitud"], hiper_df["latitud"]))  # Crear geodataframe

In [49]:
gye_quito_gdf = gpd.read_file('../../data/geojson/gye_quito.geojson')

In [50]:
gye_quito_gdf.head()

,stroke,stroke-width,stroke-opacity,fill,fill-opacity,Name,Description,geometry
0,#555555,2.0,1.0,#555555,0.5,Urbanizacion La Saiba,None,"POLYGON ((-79.89851 -2.22271, -79.89738 -2.230..."
1,#555555,2.0,1.0,#555555,0.5,Ciudadela 9 de Octubre,None,"POLYGON ((-79.89676 -2.23578, -79.89084 -2.235..."
2,#555555,2.0,1.0,#555555,0.5,Galavsa,None,"POLYGON ((-79.94596 -2.08587, -79.93931 -2.082..."
3,#555555,2.0,1.0,#555555,0.5,Parroquia Pedro Carbo,Parroquia Carbo,"POLYGON ((-79.88000 -2.18365, -79.88074 -2.183..."
4,None,NaN,NaN,None,NaN,Parroquia Olmedo,Parroquia Olmedo,"POLYGON ((-79.88402 -2.20781, -79.88352 -2.205..."


In [51]:
gdf_hiper.crs = gye_quito_gdf.crs

In [52]:
gye_quito_gdf.shape

(240, 8)

In [53]:
gdf_hiper.shape

(2443, 14)

In [54]:
gdf_hiper.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [55]:
gye_quito_gdf.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [56]:
dfgeo = gpd.sjoin(gdf_hiper,gye_quito_gdf,how = "inner", op = "within")


/home/kevinjlope/anaconda3/envs/IDS/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3337: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [57]:
dfgeo.tail()

,fecha_ingreso,grupo,cadena,comercio,estado,pais,region,provincia,ciudad,zona,...,longitud,geometry,index_right,stroke,stroke-width,stroke-opacity,fill,fill-opacity,Name,Description
2219,2019-11-19,4,4,95798,1,1.0,2.0,4.0,24.0,3.0,...,-79.945549,POINT (-79.94555 -2.07012),129,#555555,2.0,1.0,#555555,0.5,Cooperativa 5 de Diciembre,None
2237,2019-11-22,4,4,95824,1,1.0,2.0,4.0,24.0,2.0,...,-79.899513,POINT (-79.89951 -2.19750),148,#555555,2.0,1.0,#555555,0.5,Parroquia Urdaneta,None
2238,2019-11-22,4,1492,95826,1,1.0,2.0,4.0,24.0,26.0,...,-79.886891,POINT (-79.88689 -2.22068),31,#555555,2.0,1.0,#555555,0.5,Centenario,None
2247,2019-11-23,4,4,95834,1,1.0,2.0,4.0,24.0,26.0,...,-79.897726,POINT (-79.89773 -2.08128),115,#555555,2.0,1.0,#555555,0.5,Geranios 1,None
2417,2019-12-24,4,1696,96268,1,1.0,2.0,4.0,701.0,2.0,...,-79.983942,POINT (-79.98394 -2.18255),52,#555555,2.0,1.0,#555555,0.5,Portal al Sol,None


In [58]:
dfgeo.shape

(526, 22)

In [59]:
same_columns = dfgeo.columns.intersection(ventas_tri1_df.columns)

In [60]:
same_columns

Index(['grupo', 'cadena', 'comercio'], dtype='object')

In [61]:
cols_to_use = dfgeo.columns.difference(ventas_tri1_df.columns)


In [62]:
cols_to_use

Index(['Description', 'Name', 'ciudad', 'ejecutivo', 'estado', 'fecha_ingreso',
       'fill', 'fill-opacity', 'geometry', 'index_right', 'latitud',
       'longitud', 'pais', 'provincia', 'region', 'stroke', 'stroke-opacity',
       'stroke-width', 'zona'],
      dtype='object')

In [63]:
dfgeo_t1_merge = dfgeo.merge(ventas_tri1_df, on=same_columns.tolist(), how='inner')
dfgeo_t2_merge = dfgeo.merge(ventas_tri2_df, on=same_columns.tolist(), how='inner')
dfgeo_t3_merge = dfgeo.merge(ventas_tri3_df, on=same_columns.tolist(), how='inner')
dfgeo_t4_merge = dfgeo.merge(ventas_tri4_df, on=same_columns.tolist(), how='inner')
dfgeo_t1_merge.shape

(43089, 30)

In [64]:
dfgeo_t4_merge.shape

(276523, 30)

In [65]:
dfgeo_t1_merge_region_2 = dfgeo_t1_merge.loc[dfgeo_t1_merge['region'] == 2.0]
dfgeo_t2_merge_region_2 = dfgeo_t2_merge.loc[dfgeo_t2_merge['region'] == 2.0]
dfgeo_t3_merge_region_2 = dfgeo_t3_merge.loc[dfgeo_t3_merge['region'] == 2.0]
dfgeo_t4_merge_region_2 = dfgeo_t4_merge.loc[dfgeo_t4_merge['region'] == 2.0]

In [66]:
dfgeo_t1_merge_region_2.shape

(31089, 30)

In [67]:
dfgeo_region_2_concat = pd.concat([dfgeo_t1_merge_region_2, dfgeo_t2_merge_region_2, dfgeo_t3_merge_region_2, dfgeo_t4_merge_region_2])

In [68]:
dfgeo_t4_merge_region_2.shape

(230714, 30)

In [74]:
dfgeo_region_2_concat.to_csv("../../data/output/dfgeo_region_2_concat.zip", index=False, compression="zip")

In [70]:
dfgeo_region_2_concat.head()

,fecha_ingreso,grupo,cadena,comercio,estado,pais,region,provincia,ciudad,zona,...,Name,Description,fecha,hora,proveedor,producto,medio,total,cliente_final,saldo_despues
490,2019-01-03,4,4,90093,0,1.0,2.0,4.0,24.0,26.0,...,Rosales,None,2019-01-17,18:24:06,5.0,2.0,78,1.0,%..)!!.!*',8.93
491,2019-01-03,4,4,90093,0,1.0,2.0,4.0,24.0,26.0,...,Rosales,None,2019-01-25,10:12:50,83.0,2.0,78,3.0,%.!---'&.-,9.18
492,2019-01-03,4,4,90093,0,1.0,2.0,4.0,24.0,26.0,...,Rosales,None,2019-01-25,10:11:45,83.0,2.0,78,10.0,%.!-!-&.'%,18.73
493,2019-01-03,4,4,90093,0,1.0,2.0,4.0,24.0,26.0,...,Rosales,None,2019-01-21,18:11:47,5.0,15.0,78,1.0,%.-%)+%)-%,37.58
494,2019-01-03,4,4,90093,0,1.0,2.0,4.0,24.0,26.0,...,Rosales,None,2019-01-12,10:02:09,5.0,15.0,78,3.0,%..'%((*%+,34.57


In [75]:
dfgeo_region_2_concat.shape

(520081, 30)

: 

In [72]:
dfgeo_region_2_concat.isna().sum()

fecha_ingreso          0
grupo                  0
cadena                 0
comercio               0
estado                 0
pais                   0
region                 0
provincia              0
ciudad                 0
zona                   0
ejecutivo              0
latitud                0
longitud               0
geometry               0
index_right            0
stroke               861
stroke-width         861
stroke-opacity       861
fill                 861
fill-opacity         861
Name                   0
Description       517359
fecha                  0
hora                   0
proveedor              0
producto               0
medio                  0
total                  1
cliente_final          0
saldo_despues          0
dtype: int64

In [71]:
# dfgeo_t1_merge = dfgeo.merge(ventas_tri1_df, left_on=same_columns.tolist(), right_on=same_columns.tolist(), how='inner')
# dfgeo_t2_merge = dfgeo.merge(ventas_tri2_df, left_on=same_columns.tolist(), right_on=same_columns.tolist(), how='inner')
# dfgeo_t3_merge = dfgeo.merge(ventas_tri3_df, left_on=same_columns.tolist(), right_on=same_columns.tolist(), how='inner')
# dfgeo_t4_merge = dfgeo.merge(ventas_tri4_df, left_on=same_columns.tolist(), right_on=same_columns.tolist(), how='inner')

# dfgeo_t1_merge.shape
